<a href="https://colab.research.google.com/github/notmanan/Depression-Detection-Through-Multi-Modal-Data/blob/master/SVM%26RF_Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mounting Colab with GDrive
from google.colab import drive
drive.mount('/content/drive')
import os

import numpy as np
import pandas as pd 
from sklearn.ensemble import RandomForestRegressor

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_squared_log_error

import os
os.chdir('/content/drive/MyDrive/MCA Dataset')

Mounted at /content/drive


In [2]:
def makedata(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    X = np.array(X)
    X = np.average(X, axis = 0)
    X = np.array(X.reshape(1, -1))
    return X

dev = np.array(pd.read_csv('/content/drive/MyDrive/MCA Dataset/devdata.csv',delimiter=',',encoding='utf-8'))[:, 0:4]
test = np.array(pd.read_csv('/content/drive/MyDrive/MCA Dataset/testdata.csv',delimiter=',',encoding='utf-8'))[:, 0:4]
train = np.array(pd.read_csv('/content/drive/MyDrive/MCA Dataset/traindata.csv',delimiter=',',encoding='utf-8'))[:, 0:4]

X_train = np.zeros((1,74))
Y_train = []

X_test = np.zeros((1,74))
Y_test = []

for i in range(len(test)):
    data = pd.read_csv('/content/drive/My Drive/MCA Dataset/300_P/'+str(int(test[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    X_test = np.concatenate((X_test,X_temp),0)
    Y_test.append(test[i][2])
    
X_test = np.delete(X_test,0,0)
Y_test = np.array(Y_test)
count_0 = 0
count_1 = 0

for i in range(len(train)):
    data = pd.read_csv('/content/drive/My Drive/MCA Dataset/303_P/'+str(int(train[i][0]))+"_COVAREP.csv",header = None)
    if(train[i][1] == 0):
      count_0 +=1
      if(count_0<27):
        X_temp = data.iloc[:,:].values
        X_temp = makedata(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        Y_train.append(train[i][2])
    else:
      count_1 +=1
      if(count_1<27):
        X_temp = data.iloc[:,:].values
        X_temp = makedata(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        Y_train.append(train[i][2])

count_0 = 0
count_1 = 0    

for i in range(0,1):
    data = pd.read_csv('/content/drive/My Drive/MCA Dataset/302_P/'+str(int(dev[i][0]))+"_COVAREP.csv",header = None)
    if(dev[i][1] == 0):
      count_0 += 1
      if(count_0<27):
        X_temp = data.iloc[:,:].values
        X_temp = makedata(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        Y_train.append(dev[i][1])
    else:
      count_1+= 1
      if(count_1<27):
        X_temp = data.iloc[:,:].values
        X_temp = makedata(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        Y_train.append(dev[i][1])
    
X_train = np.delete(X_train,0,0)
Y_train = np.array(Y_train)

# Running Models

In [3]:
import math
class RfAudio:
  def __init__(self):
    self.classifier = RandomForestRegressor()
  
  def fitModel(self, X, Y):
    self.classifier.fit(X, Y)
  
  def predictModel(self, X):
    return self.classifier.predict(X)
  
  def scoreModel(self, yt, yp):
    return math.sqrt(mean_squared_log_error(yt, yp))

class SVMAudio:
  def __init__(self, kernel = "rbf"):
    self.classifier = SVR(kernel = kernel)
  
  def fitModel(self, X, Y):
    self.classifier.fit(X, Y)
  
  def predictModel(self, X):
    return self.classifier.predict(X)
  
  def scoreModel(self, yt, yp):
    return math.sqrt(mean_squared_log_error(yt,yp))

In [4]:
rfModel = RfAudio()
rfModel.fitModel(X_train, Y_train)
Y_pred1  = rfModel.predictModel(X_test)
print("Random forest: ")
print(rfModel.scoreModel(Y_test,Y_pred1))

svmModel = SVMAudio()
svmModel.fitModel(X_train, Y_train)
Y_pred2  = svmModel.predictModel(X_test)
print("SVM: ")
print(svmModel.scoreModel(Y_test,Y_pred2))

Random forest: 
1.1449719212813447
SVM: 
0.998649618438257
